In [15]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({'host':'192.168.250.40' , 'dbname':'IGU_LIVE_20221204','user':'sa','password':'B1admin'},)
#({'host':'192.168.1.13' , 'dbname':'CVPASARTANI','user':'sa','password':'B1admin'},)

datacompany2 = (
                {'host':'192.168.250.40' , 'dbname':'IGU_LIVE_20221204','user':'sa','password':'B1admin'},
                #{'host':'192.168.250.10' , 'dbname':'PTCKI','user':'sa','password':'B1admin'},
                #{'host':'192.168.250.10' , 'dbname':'PTSCA','user':'sa','password':'B1admin'},
                #{'host':'192.168.250.10' , 'dbname':'PTBWU','user':'sa','password':'B1admin'},
                #{'host':'192.168.250.10' , 'dbname':'PTIMS','user':'sa','password':'B1admin'},
                )


datalist=[]
 
for company in datacompany:
    msg_sql= """
    
    select  
    'Invoice' Tipe,
        b.validFor unlocked,
    a.docnum invoice,
        a.numatcard so,
        isnull(a.u_cust_PO_No,'') po_Customer,
        b.lictradnum npwp,
        a.U_IDU_FPajak ,
        a.U_Kw_No,
        f.VatGroup ,
        h.acctname,
        f.acctcode,
        a.address2,
    c.GroupName partner_group,
    f.specprice special_price,
    f.priceedit edit_price,
    a.cardcode partnercode,
    replace(replace(replace(replace(a.Address,char(13),''),char(10),''),';',''),',','') Address, 
    a.shiptocode partnername,
    b.cardname partnercompany,
    a.docdate, 
    substring(convert(varchar,a.docdate,112) ,5,2) imonth,
    left(convert(varchar,a.docdate,112) ,4) iyear,
    upper(d.SlpName) sales_in_trx,
    d.Memo slsgrp_in_trx,
    upper(e.SlpName) sales_in_master,
    e.Memo slsgrp_in_master ,
    g.itemcode ,
    '['+ g.itemcode + ']'+ g.itemname as itemname,
    g.InvntryUom uom,
    g.U_Group product_group ,
    g.U_SpeGroup product_trading,
    g.U_SubGroup product_subgroup,
    g.U_Category product_category,
    g.U_Country product_country ,
    replace(replace(g.U_Spec,char(13),''),char(10),'')  product_spec ,
    g.U_Grading product_grading,
    g.U_Brand product_brand ,
    g.U_Cutting product_cutting,
    isnull(f.U_Qty_AR,f.quantity) quantity ,
    isnull(f.U_Price_AR,f.price) price,
        f.linetotal,
        isnull(a.DiscPrcnt,0)DiscPrcnt  ,
        (f.linetotal * (isnull(a.DiscPrcnt ,0) /100  )) discAmount,
        f.linetotal - (f.linetotal * (isnull(a.DiscPrcnt ,0) /100  ))  total,
    f.GrssProfit margin 
from oinv a 
  inner join ocrd b on a.cardcode = b.cardcode 
  inner join ocrg c on b.GroupCode =c.GroupCode 
  inner join oslp d on a.slpcode = d.slpcode
  inner join oslp e on b.slpcode = e.slpcode
  inner join inv1 f on a.docentry = f.docentry 
  inner join oitm g on f.itemcode = g.itemcode 
  inner join oact h on h.acctcode = f.acctcode
where a.canceled = 'N'   and a.doctype='I'
and convert(varchar,a.DocDate,112) between '20200101' and '20221130' and f.priceedit = 'Y'

"""

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df = pd.concat(datalist)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

#master = pd.DataFrame(df, columns = ['cardcode','cardname','groupname','U_AR_Person','LicTradNum','U_SlsEmpName','Balance','CreditLine','PymntGroup','U_locktimeout','U_IGU_LockBP',])

#for index,rows in data_ar.iterrows():
#    print(rows)

#df = df.groupby("invoice").get_group(2109200391)
                
#df.fillna(0)

df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Tipe              0 non-null      object
 1   unlocked          0 non-null      object
 2   invoice           0 non-null      object
 3   so                0 non-null      object
 4   po_Customer       0 non-null      object
 5   npwp              0 non-null      object
 6   U_IDU_FPajak      0 non-null      object
 7   U_Kw_No           0 non-null      object
 8   VatGroup          0 non-null      object
 9   acctname          0 non-null      object
 10  acctcode          0 non-null      object
 11  address2          0 non-null      object
 12  partner_group     0 non-null      object
 13  special_price     0 non-null      object
 14  edit_price        0 non-null      object
 15  partnercode       0 non-null      object
 16  Address           0 non-null      object
 17  partnername       0 non-null     

,Tipe,unlocked,invoice,so,po_Customer,npwp,U_IDU_FPajak,U_Kw_No,VatGroup,acctname,...,product_grading,product_brand,product_cutting,quantity,price,linetotal,DiscPrcnt,discAmount,total,margin


In [5]:
df.to_excel ('SM0033 2020.xlsx')

In [ ]:
select  
    'Invoice' Tipe,
        b.validFor unlocked,
    a.docnum invoice,
        a.numatcard so,
        isnull(a.u_cust_PO_No,'') po_Customer,
        b.lictradnum npwp,
        a.U_IDU_FPajak ,
        a.U_Kw_No,
        f.VatGroup ,
        h.acctname,
        f.acctcode,
        a.address2,
    c.GroupName partner_group,
    f.specprice special_price,
    f.priceedit edit_price,
    a.cardcode partnercode,
    replace(replace(replace(replace(a.Address,char(13),''),char(10),''),';',''),',','') Address, 
    a.shiptocode partnername,
    b.cardname partnercompany,
    a.docdate, 
    substring(convert(varchar,a.docdate,112) ,5,2) imonth,
    left(convert(varchar,a.docdate,112) ,4) iyear,
    upper(d.SlpName) sales_in_trx,
    d.Memo slsgrp_in_trx,
    upper(e.SlpName) sales_in_master,
    e.Memo slsgrp_in_master ,
    g.itemcode ,
    '['+ g.itemcode + ']'+ g.itemname as itemname,
    g.InvntryUom uom,
    g.U_Group product_group ,
    g.U_SpeGroup product_trading,
    g.U_SubGroup product_subgroup,
    g.U_Category product_category,
    g.U_Country product_country ,
    replace(replace(g.U_Spec,char(13),''),char(10),'')  product_spec ,
    g.U_Grading product_grading,
    g.U_Brand product_brand ,
    g.U_Cutting product_cutting,
    isnull(f.U_Qty_AR,f.quantity) quantity ,
    isnull(f.U_Price_AR,f.price) price,
        f.linetotal,
        isnull(a.DiscPrcnt,0)DiscPrcnt  ,
        (f.linetotal * (isnull(a.DiscPrcnt ,0) /100  )) discAmount,
        f.linetotal - (f.linetotal * (isnull(a.DiscPrcnt ,0) /100  ))  total,
    f.GrssProfit margin 
from oinv a 
  inner join ocrd b on a.cardcode = b.cardcode 
  inner join ocrg c on b.GroupCode =c.GroupCode 
  inner join oslp d on a.slpcode = d.slpcode
  inner join oslp e on b.slpcode = e.slpcode
  inner join inv1 f on a.docentry = f.docentry 
  inner join oitm g on f.itemcode = g.itemcode 
  inner join oact h on h.acctcode = f.acctcode
where a.canceled = 'N'   and a.doctype='I'
and convert(varchar,a.DocDate,112) between '20220101' and '20220131' 
union all
select  
    'Credit Note' Tipe,
        b.validFor locked,
    a.docnum invoice,
        a.numatcard so,
        isnull(a.u_cust_PO_No,'') po_Customer,
        b.lictradnum npwp,
        a.u_FP_NO ,
        a.U_Kw_No,
        f.VatGroup ,
        h.acctname,
        f.acctcode,
        a.address2,
    c.GroupName partner_group,
    a.cardcode partnercode,
    replace(replace(replace(replace(a.Address,char(13),''),char(10),''),';',''),',','')Address, 
    a.shiptocode partnername,
    b.cardname partnercompany,
    a.docdate, 
    substring(convert(varchar,a.docdate,112) ,5,2) imonth,
    left(convert(varchar,a.docdate,112) ,4) iyear,
    d.SlpName sales_in_trx,
    d.Memo slsgrp_in_trx,
    e.SlpName sales_in_master,
    e.Memo slsgrp_in_master ,
    g.itemcode ,
    '['+ g.itemcode + ']'+ g.itemname itemname ,
    g.InvntryUom uom,
    g.U_Group product_group ,
    g.U_SpeGroup product_trading,
    g.U_SubGroup product_subgroup,
    g.U_Category product_category,
    g.U_Country product_country ,
    g.U_Spec product_spec ,
    g.U_Grading product_grading,
    g.U_Brand product_brand ,
    g.U_Cutting product_cutting,
    -1 *  f.quantity  quantity ,
      f.price  price,

        -1 * f.linetotal,
        isnull(a.DiscPrcnt ,0)DiscPrcnt,
        -1 * (f.linetotal * (isnull(a.DiscPrcnt ,0) /100  )) discAmount,
    -1* (f.linetotal -  (f.linetotal * ( isnull(a.DiscPrcnt ,0) /100 )) ) total,
    -1* f.GrssProfit margin 
from orin a 
  inner join ocrd b on a.cardcode = b.cardcode 
  inner join ocrg c on b.GroupCode =c.GroupCode 
  inner join oslp d on a.slpcode = d.slpcode
  inner join oslp e on b.slpcode = e.slpcode
  inner join rin1 f on a.docentry = f.docentry 
  inner join oitm g on f.itemcode = g.itemcode 
  inner join oact h on f.acctcode = h.acctcode
where a.canceled = 'N' and a.doctype='I'
and convert(varchar,a.DocDate,112) between '20220101' and '202201231' 